In [ ]:
#######
# load data to pandas
# train
# tune parameters on holdout set
# test
# kfolds cross validation
#######
# comparisons:
# linear regression
# decision trees (random forest via scikit learn)
# ff neural network (lin, relu, lin) higher order polyn

In [1]:
%load_ext autoreload
%autoreload 2

# Imports
import pandas as pd
from core_insure.assessor.home_assessor import HomeAssessor
from ruamel.yaml import YAML
import numpy as np
import math
from torch import nn

LABEL_KEY = 'claimAmount'

In [2]:
# Processing and training methods
def value_of_interest(value):
    if pd.isna(value):
        return False
    return True

def get_row_xy(row):
    y_label = row[LABEL_KEY] if value_of_interest(row[LABEL_KEY]) else 0
    row = row.drop([LABEL_KEY])
    rowiter = row.iteritems()
    x_array = [item[1] if value_of_interest(item[1]) else 0 for item in rowiter]
    return x_array, y_label

def get_all_xy(df):
    all_x = []
    all_y = []
    for row in df.iterrows():
        row_contents = row[1]
        x, y = get_row_xy(row_contents)
        all_x.append(x)
        all_y.append([y])
    return all_x, all_y

def train_calculate_test_loss(curr_assessor, tr_x, tr_y, te_x, te_y, v_x, v_y):
    curr_assessor.train(tr_x, tr_y, v_x, v_y)
    model = curr_assessor.model
    
    # Calculate test loss
    te_y_pred = model.eval(te_x)
    test_loss = model.loss(model._torch_var(te_y_pred), model._torch_var(te_y))
    print(f'Test Loss: {test_loss}')
    
    return test_loss

def train_test_val_split(curr_dataframe, train_percent, test_percent, val_percent):
    total_rows = curr_dataframe.shape[0]
    print(f'Total test set: {total_rows}')
    train_size = math.floor(total_rows*(train_percent/100))
    test_size = math.floor(total_rows*(test_percent/100))
    val_size = math.floor(total_rows*(val_percent/100))
    print(f'Training set: {train_size}, Testing set: {test_size}, Validation set: {val_size}')
    train_x, train_y = get_all_xy(curr_dataframe.iloc[0:train_size])
    print(f'Train set created, size: {len(train_x)}')
    last_test_index = train_size + test_size
    test_x, test_y = get_all_xy(curr_dataframe.iloc[train_size:last_test_index])
    print(f'Test set created, size: {len(test_x)}')
    last_val_index = last_test_index + val_size
    val_x, val_y = get_all_xy(curr_dataframe.iloc[last_test_index:last_val_index])
    print(f'Val set created, size: {len(val_x)}')
    return {
        'train_x': train_x,
        'train_y': train_y,
        'test_x': test_x,
        'test_y': test_y,
        'val_x': val_x,
        'val_y': val_y
    }

In [3]:
# Load data to pandas
# This dataset was downloaded from: 
# https://www.fema.gov/openfema-dataset-individual-assistance-housing-registrants-large-disasters-v1
file = open('core_insure/data/IndividualAssistanceHousingRegistrantsLargeDisasters.csv', 'r')
pd_data = pd.read_csv(file)

/Users/lisajiang/magikarp/core_insure/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Get config
default_config_file = open('core_insure/config.yaml', 'r')
yaml = YAML()
config = yaml.load(default_config_file)
print(config)

ordereddict([('base', ordereddict([('filepath', '.')])), ('data', ordereddict([('dbname', 'postgres'), ('host', 'localhost'), ('port', 5432)])), ('celery', ordereddict([('broker_url', 'amqp://localhost')])), ('assessor', ordereddict([('model', 'linear_regression'), ('linear_regression', ordereddict([('output_size', 1), ('lr', 1e-05), ('momentum', 0), ('epochs', 500)])), ('simple_nn', ordereddict([('output_size', 1), ('lr', 1e-05), ('epochs', 500), ('hidden_size', 50)])), ('filepath', '.')]))])


In [5]:
# Data processing
# Columns available
all_columns = list(pd_data.columns.values)

# calculate label: claimAmount based on repairs and replacements
pd_data = pd_data.fillna(0)
pd_data[LABEL_KEY] = pd_data['repairAmount'] + pd_data['replacementAmount']

# For now, filter on just numerical data
# Later, can convert string columns (embedding, bins, num categories)
only_numerical_data = pd_data._get_numeric_data()
sorted_numerical_df = pd_data.reindex(sorted(only_numerical_data.columns), axis=1)
numeric_columns = sorted(sorted_numerical_df.columns.values)
print(list(numeric_columns))

['censusBlockId', 'censusYear', 'claimAmount', 'damagedZipCode', 'destroyed', 'disasterNumber', 'floodDamage', 'floodInsurance', 'foundationDamage', 'foundationDamageAmount', 'grossIncome', 'habitabilityRepairsRequired', 'homeOwnersInsurance', 'householdComposition', 'inspected', 'personalPropertyEligible', 'ppfvl', 'primaryResidence', 'rentalAssistanceAmount', 'rentalAssistanceEligible', 'rentalResourceZipCode', 'repairAmount', 'repairAssistanceEligible', 'replacementAmount', 'replacementAssistanceEligible', 'roofDamage', 'roofDamageAmount', 'rpfvl', 'sbaEligible', 'specialNeeds', 'tsaCheckedIn', 'tsaEligible', 'waterLevel']


In [6]:
# remove all data where repairs were not made
nonzero_numerical_df = sorted_numerical_df.replace(0, np.nan)
nonzero_numerical_df = nonzero_numerical_df[nonzero_numerical_df[LABEL_KEY].notnull()] 
len(nonzero_numerical_df)

277242

In [7]:
# Feature Extraction
columns = [
    'roofDamageAmount',
    'foundationDamageAmount',
    'floodDamageAmount',
    'waterLevel',
    'rentalAssistanceAmount',
    'ppfvl',
    'rpfvl',
    LABEL_KEY
]
final_sorted_df_nz = pd.DataFrame(nonzero_numerical_df, columns=columns)
final_sorted_df = pd.DataFrame(sorted_numerical_df, columns=columns)

final_sorted_df_nz = final_sorted_df_nz.replace(np.nan, 0)
final_sorted_df = final_sorted_df.replace(np.nan, 0)

In [8]:
# 80-20 Split
# Full dataset of all claims
full_dataset = final_sorted_df.head(100000)
train_test_val_dataset = train_test_val_split(full_dataset, 70, 20, 10)

# Subset of only approved claims
full_dataset_nz = final_sorted_df_nz.head(100000)
train_test_val_dataset_nz = train_test_val_split(full_dataset_nz, 70, 20, 10)

Total test set: 100000
Training set: 70000, Testing set: 20000, Validation set: 10000
Train set created
Test set created
Val set created
Total test set: 100000
Training set: 70000, Testing set: 20000, Validation set: 10000
Train set created
Test set created
Val set created


In [ ]:
# Train linear regression - ALL CLAIMS
assessor_config =  config['assessor']
assessor_config['linear_regression']['epochs'] = 1000
assessor = HomeAssessor(assessor_config)

train_calculate_test_loss(assessor, 
                          train_test_val_dataset['train_x'], 
                          train_test_val_dataset['train_y'],
                          train_test_val_dataset['test_x'],
                          train_test_val_dataset['test_y'],
                          train_test_val_dataset['val_x'],
                          train_test_val_dataset['val_y'])

In [345]:
# Train linear regression - ONLY APPROVED CLAIMS
assessor_config =  config['assessor']
assessor_config['linear_regression']['epochs'] = 10000
assessor = HomeAssessor(assessor_config)

train_calculate_test_loss(assessor, 
                          train_test_val_dataset_nz['train_x'], 
                          train_test_val_dataset_nz['train_y'],
                          train_test_val_dataset_nz['test_x'],
                          train_test_val_dataset_nz['test_y'],
                          train_test_val_dataset_nz['val_x'],
                          train_test_val_dataset_nz['val_y'])

Epoch 0, Train_Loss: 6272.978515625, Val_Loss: 6690.60498046875
Epoch 1, Train_Loss: 6253.2080078125, Val_Loss: 6669.0888671875
Epoch 2, Train_Loss: 6233.55712890625, Val_Loss: 6647.59912109375
Epoch 3, Train_Loss: 6213.60986328125, Val_Loss: 6626.13916015625
Epoch 4, Train_Loss: 6193.828125, Val_Loss: 6604.6171875
Epoch 5, Train_Loss: 6174.05224609375, Val_Loss: 6583.150390625
Epoch 6, Train_Loss: 6154.3896484375, Val_Loss: 6561.6767578125
Epoch 7, Train_Loss: 6134.6845703125, Val_Loss: 6540.20068359375
Epoch 8, Train_Loss: 6114.9873046875, Val_Loss: 6518.74755859375
Epoch 9, Train_Loss: 6095.2724609375, Val_Loss: 6497.275390625
Epoch 10, Train_Loss: 6075.57080078125, Val_Loss: 6475.81982421875
Epoch 11, Train_Loss: 6055.857421875, Val_Loss: 6454.37744140625
Epoch 12, Train_Loss: 6036.18212890625, Val_Loss: 6432.93017578125
Epoch 13, Train_Loss: 6016.34814453125, Val_Loss: 6411.44580078125
Epoch 14, Train_Loss: 5996.703125, Val_Loss: 6390.01416015625
Epoch 15, Train_Loss: 5976.8789062

Epoch 130, Train_Loss: 4350.6474609375, Val_Loss: 4549.7392578125
Epoch 131, Train_Loss: 4341.63818359375, Val_Loss: 4539.11181640625
Epoch 132, Train_Loss: 4332.82177734375, Val_Loss: 4528.60888671875
Epoch 133, Train_Loss: 4324.08154296875, Val_Loss: 4518.2158203125
Epoch 134, Train_Loss: 4315.4990234375, Val_Loss: 4507.99853515625
Epoch 135, Train_Loss: 4307.0302734375, Val_Loss: 4497.87939453125
Epoch 136, Train_Loss: 4298.49853515625, Val_Loss: 4487.76953125
Epoch 137, Train_Loss: 4290.06298828125, Val_Loss: 4477.68603515625
Epoch 138, Train_Loss: 4281.66845703125, Val_Loss: 4467.65966796875
Epoch 139, Train_Loss: 4273.5029296875, Val_Loss: 4457.84912109375
Epoch 140, Train_Loss: 4265.4072265625, Val_Loss: 4448.15576171875
Epoch 141, Train_Loss: 4257.4521484375, Val_Loss: 4438.63623046875
Epoch 142, Train_Loss: 4249.44580078125, Val_Loss: 4429.15771484375
Epoch 143, Train_Loss: 4241.6171875, Val_Loss: 4419.82177734375
Epoch 144, Train_Loss: 4233.9990234375, Val_Loss: 4410.63037109

Epoch 253, Train_Loss: 3971.524658203125, Val_Loss: 4085.703125
Epoch 254, Train_Loss: 3971.494873046875, Val_Loss: 4085.59033203125
Epoch 255, Train_Loss: 3971.42041015625, Val_Loss: 4085.552490234375
Epoch 256, Train_Loss: 3971.424072265625, Val_Loss: 4085.4755859375
Epoch 257, Train_Loss: 3971.36767578125, Val_Loss: 4085.415283203125
Epoch 258, Train_Loss: 3971.361328125, Val_Loss: 4085.3408203125
Epoch 259, Train_Loss: 3971.33935546875, Val_Loss: 4085.2548828125
Epoch 260, Train_Loss: 3971.31982421875, Val_Loss: 4085.222412109375
Epoch 261, Train_Loss: 3971.2685546875, Val_Loss: 4085.184814453125
Epoch 262, Train_Loss: 3971.259765625, Val_Loss: 4085.1484375
Epoch 263, Train_Loss: 3971.237548828125, Val_Loss: 4085.113525390625
Epoch 264, Train_Loss: 3971.2060546875, Val_Loss: 4085.05029296875
Epoch 265, Train_Loss: 3971.180419921875, Val_Loss: 4085.031982421875
Epoch 266, Train_Loss: 3971.1884765625, Val_Loss: 4085.0263671875
Epoch 267, Train_Loss: 3971.177978515625, Val_Loss: 4084.

Epoch 378, Train_Loss: 3970.697998046875, Val_Loss: 4084.138916015625
Epoch 379, Train_Loss: 3970.6953125, Val_Loss: 4084.13720703125
Epoch 380, Train_Loss: 3970.69482421875, Val_Loss: 4084.114501953125
Epoch 381, Train_Loss: 3970.692626953125, Val_Loss: 4084.110107421875
Epoch 382, Train_Loss: 3970.689453125, Val_Loss: 4084.113525390625
Epoch 383, Train_Loss: 3970.6875, Val_Loss: 4084.102294921875
Epoch 384, Train_Loss: 3970.6865234375, Val_Loss: 4084.09765625
Epoch 385, Train_Loss: 3970.678955078125, Val_Loss: 4084.0908203125
Epoch 386, Train_Loss: 3970.67431640625, Val_Loss: 4084.089599609375
Epoch 387, Train_Loss: 3970.671142578125, Val_Loss: 4084.080322265625
Epoch 388, Train_Loss: 3970.6669921875, Val_Loss: 4084.078369140625
Epoch 389, Train_Loss: 3970.664794921875, Val_Loss: 4084.07470703125
Epoch 390, Train_Loss: 3970.66064453125, Val_Loss: 4084.072021484375
Epoch 391, Train_Loss: 3970.6591796875, Val_Loss: 4084.073974609375
Epoch 392, Train_Loss: 3970.65087890625, Val_Loss: 40

Epoch 503, Train_Loss: 3969.822998046875, Val_Loss: 4083.3095703125
Epoch 504, Train_Loss: 3969.76513671875, Val_Loss: 4083.2900390625
Epoch 505, Train_Loss: 3969.802490234375, Val_Loss: 4083.264892578125
Epoch 506, Train_Loss: 3969.667236328125, Val_Loss: 4083.246826171875
Epoch 507, Train_Loss: 3969.67822265625, Val_Loss: 4083.251953125
Epoch 508, Train_Loss: 3969.67138671875, Val_Loss: 4083.239501953125
Epoch 509, Train_Loss: 3969.66259765625, Val_Loss: 4083.23486328125
Epoch 510, Train_Loss: 3969.658447265625, Val_Loss: 4083.2294921875
Epoch 511, Train_Loss: 3969.66259765625, Val_Loss: 4083.23193359375
Epoch 512, Train_Loss: 3969.651611328125, Val_Loss: 4083.219482421875
Epoch 513, Train_Loss: 3969.6357421875, Val_Loss: 4083.211181640625
Epoch 514, Train_Loss: 3969.62548828125, Val_Loss: 4083.216064453125
Epoch 515, Train_Loss: 3969.583984375, Val_Loss: 4083.21728515625
Epoch 516, Train_Loss: 3969.703369140625, Val_Loss: 4083.226806640625
Epoch 517, Train_Loss: 3969.731689453125, V

...

Epoch 9494, Train_Loss: 3798.22216796875, Val_Loss: 3934.29638671875
Epoch 9495, Train_Loss: 3798.261474609375, Val_Loss: 3934.28955078125
Epoch 9496, Train_Loss: 3798.214111328125, Val_Loss: 3934.306884765625
Epoch 9497, Train_Loss: 3798.212890625, Val_Loss: 3934.295166015625
Epoch 9498, Train_Loss: 3798.275390625, Val_Loss: 3934.328857421875
Epoch 9499, Train_Loss: 3798.232421875, Val_Loss: 3934.283935546875
Epoch 9500, Train_Loss: 3798.32177734375, Val_Loss: 3934.335205078125
Epoch 9501, Train_Loss: 3798.228515625, Val_Loss: 3934.279296875
Epoch 9502, Train_Loss: 3798.269287109375, Val_Loss: 3934.3212890625
Epoch 9503, Train_Loss: 3798.258544921875, Val_Loss: 3934.27392578125
Epoch 9504, Train_Loss: 3798.258544921875, Val_Loss: 3934.279296875
Epoch 9505, Train_Loss: 3798.24267578125, Val_Loss: 3934.277099609375
Epoch 9506, Train_Loss: 3798.24072265625, Val_Loss: 3934.2744140625
Epoch 9507, Train_Loss: 3798.25, Val_Loss: 3934.279296875
Epoch 9508, Train_Loss: 3798.2470703125, Val_Los

Epoch 9616, Train_Loss: 3798.1220703125, Val_Loss: 3934.075927734375
Epoch 9617, Train_Loss: 3798.1220703125, Val_Loss: 3934.21044921875
Epoch 9618, Train_Loss: 3798.159423828125, Val_Loss: 3934.1171875
Epoch 9619, Train_Loss: 3798.116455078125, Val_Loss: 3934.06640625
Epoch 9620, Train_Loss: 3798.1572265625, Val_Loss: 3934.193603515625
Epoch 9621, Train_Loss: 3798.146484375, Val_Loss: 3934.134521484375
Epoch 9622, Train_Loss: 3798.1298828125, Val_Loss: 3934.02685546875
Epoch 9623, Train_Loss: 3798.162841796875, Val_Loss: 3934.232421875
Epoch 9624, Train_Loss: 3798.163330078125, Val_Loss: 3934.162109375
Epoch 9625, Train_Loss: 3798.09521484375, Val_Loss: 3933.96435546875
Epoch 9626, Train_Loss: 3798.191650390625, Val_Loss: 3934.890869140625
Epoch 9627, Train_Loss: 3798.1220703125, Val_Loss: 3934.0205078125
Epoch 9628, Train_Loss: 3798.19873046875, Val_Loss: 3933.69677734375
Epoch 9629, Train_Loss: 3798.25732421875, Val_Loss: 3935.064453125
Epoch 9630, Train_Loss: 3798.14453125, Val_Los

Epoch 9738, Train_Loss: 3797.94091796875, Val_Loss: 3934.15234375
Epoch 9739, Train_Loss: 3797.9189453125, Val_Loss: 3934.264404296875
Epoch 9740, Train_Loss: 3798.003662109375, Val_Loss: 3933.7958984375
Epoch 9741, Train_Loss: 3797.9296875, Val_Loss: 3934.191162109375
Epoch 9742, Train_Loss: 3797.9228515625, Val_Loss: 3934.160888671875
Epoch 9743, Train_Loss: 3798.03466796875, Val_Loss: 3933.800048828125
Epoch 9744, Train_Loss: 3798.013427734375, Val_Loss: 3934.268310546875
Epoch 9745, Train_Loss: 3797.929443359375, Val_Loss: 3933.99951171875
Epoch 9746, Train_Loss: 3798.044677734375, Val_Loss: 3933.681640625
Epoch 9747, Train_Loss: 3798.008544921875, Val_Loss: 3934.269287109375
Epoch 9748, Train_Loss: 3797.922607421875, Val_Loss: 3934.134521484375
Epoch 9749, Train_Loss: 3798.019287109375, Val_Loss: 3933.742431640625
Epoch 9750, Train_Loss: 3797.906982421875, Val_Loss: 3934.185546875
Epoch 9751, Train_Loss: 3797.9111328125, Val_Loss: 3934.20068359375
Epoch 9752, Train_Loss: 3798.0039

Epoch 9860, Train_Loss: 3797.745361328125, Val_Loss: 3933.782470703125
Epoch 9861, Train_Loss: 3797.6611328125, Val_Loss: 3933.84033203125
Epoch 9862, Train_Loss: 3797.7578125, Val_Loss: 3933.73193359375
Epoch 9863, Train_Loss: 3797.739501953125, Val_Loss: 3933.7412109375
Epoch 9864, Train_Loss: 3797.6806640625, Val_Loss: 3933.79443359375
Epoch 9865, Train_Loss: 3797.660888671875, Val_Loss: 3933.81884765625
Epoch 9866, Train_Loss: 3797.738037109375, Val_Loss: 3933.728515625
Epoch 9867, Train_Loss: 3797.72998046875, Val_Loss: 3933.73876953125
Epoch 9868, Train_Loss: 3797.652587890625, Val_Loss: 3933.83154296875
Epoch 9869, Train_Loss: 3797.660400390625, Val_Loss: 3933.74560546875
Epoch 9870, Train_Loss: 3797.659912109375, Val_Loss: 3933.74072265625
Epoch 9871, Train_Loss: 3797.675048828125, Val_Loss: 3933.797119140625
Epoch 9872, Train_Loss: 3797.6591796875, Val_Loss: 3933.763916015625
Epoch 9873, Train_Loss: 3797.670654296875, Val_Loss: 3933.753173828125
Epoch 9874, Train_Loss: 3797.65

Epoch 9983, Train_Loss: 3797.533447265625, Val_Loss: 3933.580078125
Epoch 9984, Train_Loss: 3797.528564453125, Val_Loss: 3933.580078125
Epoch 9985, Train_Loss: 3797.529541015625, Val_Loss: 3933.578369140625
Epoch 9986, Train_Loss: 3797.54150390625, Val_Loss: 3933.57763671875
Epoch 9987, Train_Loss: 3797.528076171875, Val_Loss: 3933.576416015625
Epoch 9988, Train_Loss: 3797.5400390625, Val_Loss: 3933.576904296875
Epoch 9989, Train_Loss: 3797.523681640625, Val_Loss: 3933.57958984375
Epoch 9990, Train_Loss: 3797.5234375, Val_Loss: 3933.577880859375
Epoch 9991, Train_Loss: 3797.519287109375, Val_Loss: 3933.5791015625
Epoch 9992, Train_Loss: 3797.524169921875, Val_Loss: 3933.573974609375
Epoch 9993, Train_Loss: 3797.52392578125, Val_Loss: 3933.5732421875
Epoch 9994, Train_Loss: 3797.52099609375, Val_Loss: 3933.575927734375
Epoch 9995, Train_Loss: 3797.54736328125, Val_Loss: 3933.572509765625
Epoch 9996, Train_Loss: 3797.519775390625, Val_Loss: 3933.57275390625
Epoch 9997, Train_Loss: 3797.5

In [ ]:
# Train nn module -- ALL CLAIMS
assessor_config =  config['assessor']
assessor_config['simple_nn']['epochs'] = 10
assessor_config['model'] = 'simple_nn'
assessor = HomeAssessor(assessor_config)

train_calculate_test_loss(assessor, 
                          train_test_val_dataset['train_x'], 
                          train_test_val_dataset['train_y'],
                          train_test_val_dataset['test_x'],
                          train_test_val_dataset['test_y'],
                          train_test_val_dataset['val_x'],
                          train_test_val_dataset['val_y'])

In [13]:
# Train nn module -- ONLY APPROVED CLAIMS
assessor_config =  config['assessor']
assessor_config['simple_nn']['epochs'] = 10000
assessor_config['model'] = 'simple_nn'
assessor = HomeAssessor(assessor_config)

train_calculate_test_loss(assessor, 
                          train_test_val_dataset_nz['train_x'], 
                          train_test_val_dataset_nz['train_y'],
                          train_test_val_dataset_nz['test_x'],
                          train_test_val_dataset_nz['test_y'],
                          train_test_val_dataset_nz['val_x'],
                          train_test_val_dataset_nz['val_y'])

Epoch 0, Train_Loss: 5810.2001953125, Val_Loss: 6186.708984375
Epoch 1, Train_Loss: 5809.87939453125, Val_Loss: 6186.34228515625
Epoch 2, Train_Loss: 5809.5517578125, Val_Loss: 6185.9658203125
Epoch 3, Train_Loss: 5809.203125, Val_Loss: 6185.56689453125
Epoch 4, Train_Loss: 5808.87255859375, Val_Loss: 6185.22021484375
Epoch 5, Train_Loss: 5808.48291015625, Val_Loss: 6184.8447265625
Epoch 6, Train_Loss: 5808.1611328125, Val_Loss: 6184.48779296875
Epoch 7, Train_Loss: 5807.81689453125, Val_Loss: 6184.1279296875
Epoch 8, Train_Loss: 5807.48681640625, Val_Loss: 6183.7490234375
Epoch 9, Train_Loss: 5807.169921875, Val_Loss: 6183.3857421875
Epoch 10, Train_Loss: 5806.86376953125, Val_Loss: 6183.0283203125
Epoch 11, Train_Loss: 5806.54736328125, Val_Loss: 6182.66064453125
Epoch 12, Train_Loss: 5806.18994140625, Val_Loss: 6182.29541015625
Epoch 13, Train_Loss: 5805.8603515625, Val_Loss: 6181.93310546875
Epoch 14, Train_Loss: 5805.53076171875, Val_Loss: 6181.548828125
Epoch 15, Train_Loss: 5805

Epoch 131, Train_Loss: 5766.67138671875, Val_Loss: 6139.04296875
Epoch 132, Train_Loss: 5766.3447265625, Val_Loss: 6138.67822265625
Epoch 133, Train_Loss: 5766.025390625, Val_Loss: 6138.32568359375
Epoch 134, Train_Loss: 5765.69091796875, Val_Loss: 6137.96435546875
Epoch 135, Train_Loss: 5765.392578125, Val_Loss: 6137.5947265625
Epoch 136, Train_Loss: 5765.0517578125, Val_Loss: 6137.2294921875
Epoch 137, Train_Loss: 5764.728515625, Val_Loss: 6136.8671875
Epoch 138, Train_Loss: 5764.408203125, Val_Loss: 6136.5244140625
Epoch 139, Train_Loss: 5763.9931640625, Val_Loss: 6136.16552734375
Epoch 140, Train_Loss: 5763.673828125, Val_Loss: 6135.77099609375
Epoch 141, Train_Loss: 5763.36572265625, Val_Loss: 6135.4189453125
Epoch 142, Train_Loss: 5763.04052734375, Val_Loss: 6135.044921875
Epoch 143, Train_Loss: 5762.7216796875, Val_Loss: 6134.69384765625
Epoch 144, Train_Loss: 5762.3935546875, Val_Loss: 6134.33349609375
Epoch 145, Train_Loss: 5762.044921875, Val_Loss: 6133.96875
Epoch 146, Train

Epoch 256, Train_Loss: 5725.85009765625, Val_Loss: 6094.12158203125
Epoch 257, Train_Loss: 5725.5244140625, Val_Loss: 6093.77392578125
Epoch 258, Train_Loss: 5725.21728515625, Val_Loss: 6093.40966796875
Epoch 259, Train_Loss: 5724.89404296875, Val_Loss: 6093.05517578125
Epoch 260, Train_Loss: 5724.58642578125, Val_Loss: 6092.71875
Epoch 261, Train_Loss: 5724.173828125, Val_Loss: 6092.3388671875
Epoch 262, Train_Loss: 5723.86181640625, Val_Loss: 6091.98193359375
Epoch 263, Train_Loss: 5723.54931640625, Val_Loss: 6091.65234375
Epoch 264, Train_Loss: 5723.236328125, Val_Loss: 6091.27783203125
Epoch 265, Train_Loss: 5722.89013671875, Val_Loss: 6090.91259765625
Epoch 266, Train_Loss: 5722.57861328125, Val_Loss: 6090.5673828125
Epoch 267, Train_Loss: 5722.259765625, Val_Loss: 6090.2099609375
Epoch 268, Train_Loss: 5721.94921875, Val_Loss: 6089.8583984375
Epoch 269, Train_Loss: 5721.64306640625, Val_Loss: 6089.49951171875
Epoch 270, Train_Loss: 5721.3125, Val_Loss: 6089.12255859375
Epoch 271,

Epoch 382, Train_Loss: 5685.18798828125, Val_Loss: 6049.4501953125
Epoch 383, Train_Loss: 5684.85546875, Val_Loss: 6049.10302734375
Epoch 384, Train_Loss: 5684.55810546875, Val_Loss: 6048.7626953125
Epoch 385, Train_Loss: 5684.22607421875, Val_Loss: 6048.42138671875
Epoch 386, Train_Loss: 5683.90380859375, Val_Loss: 6048.05078125
Epoch 387, Train_Loss: 5683.58740234375, Val_Loss: 6047.7080078125
Epoch 388, Train_Loss: 5683.28173828125, Val_Loss: 6047.35498046875
Epoch 389, Train_Loss: 5682.9765625, Val_Loss: 6047.00048828125
Epoch 390, Train_Loss: 5682.66015625, Val_Loss: 6046.65771484375
Epoch 391, Train_Loss: 5682.3642578125, Val_Loss: 6046.306640625
Epoch 392, Train_Loss: 5682.0400390625, Val_Loss: 6045.95166015625
Epoch 393, Train_Loss: 5681.74658203125, Val_Loss: 6045.6025390625
Epoch 394, Train_Loss: 5681.4228515625, Val_Loss: 6045.25341796875
Epoch 395, Train_Loss: 5681.0302734375, Val_Loss: 6044.890625
Epoch 396, Train_Loss: 5680.71630859375, Val_Loss: 6044.5498046875
Epoch 397

Epoch 510, Train_Loss: 5644.2763671875, Val_Loss: 6004.65576171875
Epoch 511, Train_Loss: 5643.94287109375, Val_Loss: 6004.3037109375
Epoch 512, Train_Loss: 5643.6259765625, Val_Loss: 6003.9482421875
Epoch 513, Train_Loss: 5643.31103515625, Val_Loss: 6003.60791015625
Epoch 514, Train_Loss: 5643.0068359375, Val_Loss: 6003.2529296875
Epoch 515, Train_Loss: 5642.71142578125, Val_Loss: 6002.89501953125
Epoch 516, Train_Loss: 5642.3984375, Val_Loss: 6002.55029296875
Epoch 517, Train_Loss: 5642.09423828125, Val_Loss: 6002.2099609375
Epoch 518, Train_Loss: 5641.74462890625, Val_Loss: 6001.869140625
Epoch 519, Train_Loss: 5641.4541015625, Val_Loss: 6001.537109375
Epoch 520, Train_Loss: 5641.13427734375, Val_Loss: 6001.1591796875
Epoch 521, Train_Loss: 5640.8212890625, Val_Loss: 6000.79638671875
Epoch 522, Train_Loss: 5640.52490234375, Val_Loss: 6000.45703125
Epoch 523, Train_Loss: 5640.2275390625, Val_Loss: 6000.10986328125
Epoch 524, Train_Loss: 5639.9248046875, Val_Loss: 5999.7607421875
Epoc

...

Epoch 9521, Train_Loss: 4060.3720703125, Val_Loss: 4197.09228515625
Epoch 9522, Train_Loss: 4060.27880859375, Val_Loss: 4197.02490234375
Epoch 9523, Train_Loss: 4060.219970703125, Val_Loss: 4196.9462890625
Epoch 9524, Train_Loss: 4060.153564453125, Val_Loss: 4196.88037109375
Epoch 9525, Train_Loss: 4060.09912109375, Val_Loss: 4196.81640625
Epoch 9526, Train_Loss: 4060.04296875, Val_Loss: 4196.72900390625
Epoch 9527, Train_Loss: 4059.974365234375, Val_Loss: 4196.6494140625
Epoch 9528, Train_Loss: 4059.9228515625, Val_Loss: 4196.58447265625
Epoch 9529, Train_Loss: 4059.881103515625, Val_Loss: 4196.50927734375
Epoch 9530, Train_Loss: 4059.817626953125, Val_Loss: 4196.4521484375
Epoch 9531, Train_Loss: 4059.7578125, Val_Loss: 4196.38623046875
Epoch 9532, Train_Loss: 4059.695068359375, Val_Loss: 4196.31494140625
Epoch 9533, Train_Loss: 4059.624267578125, Val_Loss: 4196.22900390625
Epoch 9534, Train_Loss: 4059.5693359375, Val_Loss: 4196.1591796875
Epoch 9535, Train_Loss: 4059.508056640625, V

Epoch 9645, Train_Loss: 4053.122802734375, Val_Loss: 4188.42919921875
Epoch 9646, Train_Loss: 4053.071533203125, Val_Loss: 4188.3603515625
Epoch 9647, Train_Loss: 4052.957275390625, Val_Loss: 4188.3046875
Epoch 9648, Train_Loss: 4052.907958984375, Val_Loss: 4188.23046875
Epoch 9649, Train_Loss: 4052.84716796875, Val_Loss: 4188.16748046875
Epoch 9650, Train_Loss: 4052.792724609375, Val_Loss: 4188.0986328125
Epoch 9651, Train_Loss: 4052.7470703125, Val_Loss: 4188.017578125
Epoch 9652, Train_Loss: 4052.68017578125, Val_Loss: 4187.96142578125
Epoch 9653, Train_Loss: 4052.626220703125, Val_Loss: 4187.8857421875
Epoch 9654, Train_Loss: 4052.5517578125, Val_Loss: 4187.8232421875
Epoch 9655, Train_Loss: 4052.48779296875, Val_Loss: 4187.76904296875
Epoch 9656, Train_Loss: 4052.43701171875, Val_Loss: 4187.69775390625
Epoch 9657, Train_Loss: 4052.40966796875, Val_Loss: 4187.6240234375
Epoch 9658, Train_Loss: 4052.40869140625, Val_Loss: 4187.5478515625
Epoch 9659, Train_Loss: 4052.355224609375, Va

Epoch 9769, Train_Loss: 4046.349609375, Val_Loss: 4180.2919921875
Epoch 9770, Train_Loss: 4046.30859375, Val_Loss: 4180.23095703125
Epoch 9771, Train_Loss: 4046.25244140625, Val_Loss: 4180.1669921875
Epoch 9772, Train_Loss: 4046.19482421875, Val_Loss: 4180.095703125
Epoch 9773, Train_Loss: 4046.126708984375, Val_Loss: 4180.0439453125
Epoch 9774, Train_Loss: 4046.069091796875, Val_Loss: 4179.98681640625
Epoch 9775, Train_Loss: 4046.010986328125, Val_Loss: 4179.91796875
Epoch 9776, Train_Loss: 4045.96240234375, Val_Loss: 4179.86181640625
Epoch 9777, Train_Loss: 4045.968505859375, Val_Loss: 4179.77978515625
Epoch 9778, Train_Loss: 4045.924072265625, Val_Loss: 4179.7265625
Epoch 9779, Train_Loss: 4045.859130859375, Val_Loss: 4179.65625
Epoch 9780, Train_Loss: 4045.806640625, Val_Loss: 4179.5927734375
Epoch 9781, Train_Loss: 4045.76171875, Val_Loss: 4179.5380859375
Epoch 9782, Train_Loss: 4045.708251953125, Val_Loss: 4179.484375
Epoch 9783, Train_Loss: 4045.66455078125, Val_Loss: 4179.41406

Epoch 9894, Train_Loss: 4039.926025390625, Val_Loss: 4172.48388671875
Epoch 9895, Train_Loss: 4039.87939453125, Val_Loss: 4172.42578125
Epoch 9896, Train_Loss: 4039.874755859375, Val_Loss: 4172.359375
Epoch 9897, Train_Loss: 4039.8291015625, Val_Loss: 4172.29345703125
Epoch 9898, Train_Loss: 4039.767822265625, Val_Loss: 4172.23876953125
Epoch 9899, Train_Loss: 4039.589111328125, Val_Loss: 4172.16943359375
Epoch 9900, Train_Loss: 4039.537841796875, Val_Loss: 4172.12939453125
Epoch 9901, Train_Loss: 4039.496337890625, Val_Loss: 4172.07275390625
Epoch 9902, Train_Loss: 4039.445068359375, Val_Loss: 4172.009765625
Epoch 9903, Train_Loss: 4039.396484375, Val_Loss: 4171.939453125
Epoch 9904, Train_Loss: 4039.353515625, Val_Loss: 4171.87060546875
Epoch 9905, Train_Loss: 4039.32080078125, Val_Loss: 4171.81103515625
Epoch 9906, Train_Loss: 4039.28173828125, Val_Loss: 4171.751953125
Epoch 9907, Train_Loss: 4039.2314453125, Val_Loss: 4171.6982421875
Epoch 9908, Train_Loss: 4039.12353515625, Val_Lo

tensor(3936.1963)